In [1]:
import sqlite3
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Load embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Define semantic concept vector
math_concept = "mathematical concept"
math_vector = model.encode([math_concept])


/share/u/koyena/miniconda3/envs/newmats/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [ ]:
# Connect to your DB
conn = sqlite3.connect("data/autointerp.db")  # change this to your file
df = pd.read_sql_query("SELECT label, indices FROM autointerp", conn)

# Generate embeddings for all labels
label_embeddings = model.encode(df['label'].tolist())

In [4]:

# Compute similarity with the "math" concept
similarities = cosine_similarity(label_embeddings, math_vector)

In [7]:

# Add similarity scores to DataFrame
df['similarity_to_math'] = similarities[:, 0]

# Set a threshold for "math-related" (you can tune this!)
math_df = df[df['similarity_to_math'] > 0.4]  # try 0.4–0.6 and adjust as needed

# Show label and indices
print(math_df[['label', 'indices']])

                                                   label  \
7       Mathematical calculation and numerical reasoning   
10                     Mathematical constraint reasoning   
14     Code assignment operations in mathematical con...   
27     Numbers written on physical objects in problem...   
41           Beginning of LaTeX mathematical expressions   
...                                                  ...   
23423       Mathematical notation and programming syntax   
23432             Programming and algorithm explanations   
23437       Algorithmic and mathematical problem-solving   
23468                 Mathematical optimization problems   
23521                   Mathematical addition operations   

                                                 indices  
7                                                     []  
10                                                    []  
14                                                    []  
27                                         

In [8]:
math_df[['label', 'indices']].to_csv("r1_math_features.csv")